In [2]:
import numpy as np
import os
import shutil
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    try:
        for device in gpu_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("✅ GPU is enabled and will be used for training.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU found, training will use CPU.")

✅ GPU is enabled and will be used for training.


In [4]:
DATASET_1 = "/kaggle/input/covid19-lung-ct-scans/COVID-19_Lung_CT_Scans"
DATASET_2 = "/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset"

In [5]:
DATASET_1 = "/kaggle/input/covid19-lung-ct-scans/COVID-19_Lung_CT_Scans"
DATASET_2 = "/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset"
COMBINED_DATASET = "/kaggle/working/combined_dataset"

In [6]:
if not os.path.exists(COMBINED_DATASET):
    os.makedirs(COMBINED_DATASET)

In [7]:
categories = {
    "COVID-19": "COVID-19", "Non-COVID-19": "Non-COVID-19",  # CT Scan Dataset
    "COVID": "COVID-19", "Lung_Opacity": "Lung_Opacity", "Normal": "Normal", "Viral Pneumonia": "Viral Pneumonia"  # Radiography Dataset
}

In [8]:
for category in ["COVID-19", "Non-COVID-19"]:
    source_path = os.path.join(DATASET_1, category)
    target_path = os.path.join(COMBINED_DATASET, categories[category])
    os.makedirs(target_path, exist_ok=True)
    for file in os.listdir(source_path):
        shutil.copy(os.path.join(source_path, file), target_path)


In [9]:
for category in ["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]:
    source_path = os.path.join(DATASET_2, category, "images")
    target_path = os.path.join(COMBINED_DATASET, categories[category])
    os.makedirs(target_path, exist_ok=True)
    for file in os.listdir(source_path):
        shutil.copy(os.path.join(source_path, file), target_path)

print("✅ Dataset successfully merged into:", COMBINED_DATASET)

✅ Dataset successfully merged into: /kaggle/working/combined_dataset


In [10]:
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [11]:
train_generator = datagen.flow_from_directory(
    COMBINED_DATASET, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training'
)
val_generator = datagen.flow_from_directory(
    COMBINED_DATASET, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation'
)

Found 23685 images belonging to 5 classes.
Found 5919 images belonging to 5 classes.


In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
epochs = 12
with tf.device('/GPU:0'):
    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)


Epoch 1/12


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


741/741 ━━━━━━━━━━━━━━━━━━━━ 145s 176ms/step - accuracy: 0.7375 - loss: 0.7345 - val_accuracy: 0.8566 - val_loss: 0.5138
Epoch 2/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 167ms/step - accuracy: 0.8106 - loss: 0.4676 - val_accuracy: 0.8824 - val_loss: 0.3878
Epoch 3/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 124s 166ms/step - accuracy: 0.8366 - loss: 0.4087 - val_accuracy: 0.9071 - val_loss: 0.3089
Epoch 4/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 168ms/step - accuracy: 0.8400 - loss: 0.3857 - val_accuracy: 0.8789 - val_loss: 0.3992
Epoch 5/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 168ms/step - accuracy: 0.8494 - loss: 0.3552 - val_accuracy: 0.9061 - val_loss: 0.3498
Epoch 6/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 167ms/step - accuracy: 0.8685 - loss: 0.3305 - val_accuracy: 0.9040 - val_loss: 0.3200
Epoch 7/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 167ms/step - accuracy: 0.8775 - loss: 0.3047 - val_accuracy: 0.8944 - val_loss: 0.3749
Epoch 8/12
741/741 ━━━━━━━━━━━━━━━━━━━━ 125s 168ms/step - accuracy: 0.8798 - loss: 0.29

In [16]:
test_loss, test_acc = model.evaluate(val_generator)
print(f"🎯 Test Accuracy: {test_acc:.4f}")

185/185 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step - accuracy: 0.9192 - loss: 0.3670
🎯 Test Accuracy: 0.9078


In [17]:
model.save("/kaggle/working/models/covid19_vgg16_combined.h5")